In [1]:
import cv2
import numpy as np  
import pygetwindow as gw  
import mss
import time
from flask import Flask, request, jsonify  
import json  
import requests
import tkinter as tk  
from websocket_server import WebsocketServer  
import threading  
import psutil
import win32process

In [ ]:
## Định nghĩa màu light green  
lower_light_green = np.array([40, 150, 50])  
upper_light_green = np.array([80, 255, 255])  

size = '950x530'

# Xác định vùng cần nhận diện trong giao diện game  
roi1_x, roi1_y, roi1_width, roi1_height = 27, 10, 40, 100   
roi2_x, roi2_y, roi2_width, roi2_height = 97, 10, 40, 100 

class ServerManager:  
    def __init__(self, root):  
        self.root = root  
        self.root.title("WebSocket Server Manager")  
        self.servers = {}  
        self.open_windows = {}  # Track open windows
        
        self.port_label = tk.Label(root, text="Enter Port:")  
        self.port_label.pack()  

        self.port_entry = tk.Entry(root, width=30)  
        self.port_entry.pack()  

        self.add_button = tk.Button(root, text="Add Server", width=30, command=self.add_server)  
        self.add_button.pack()  

        self.server_frame = tk.Frame(root)  
        self.server_frame.pack(pady=10)  

        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)  

    def add_server(self):  
        port = self.port_entry.get()  
        if port and port.isdigit():  
            port = int(port)  
            if port not in self.servers:  
                server = WebsocketServer(host='0.0.0.0', port=port)  

                server.set_fn_new_client(self.new_client)  
                server.set_fn_client_left(self.client_left)  
                server.set_fn_message_received(self.message_received)  

                threading.Thread(target=server.run_forever, daemon=True).start()  
                self.servers[port] = {'server': server, 'clients': {}}  
                self.update_server_list()  
                self.port_entry.delete(0, tk.END)  

    def new_client(self, client, server):  
        port = server.port  
        self.servers[port]['clients'][client['id']] = None  
        self.update_server_list()  

    def client_left(self, client, server):  
        port = server.port  
        if client['id'] in self.servers[port]['clients']:  
            del self.servers[port]['clients'][client['id']]  
        self.update_server_list()  

    def message_received(self, client, server, message):  
        port = server.port  
        self.servers[port]['clients'][client['id']] = message  # Assume the message is the username  
        self.update_server_list()  

    def update_server_list(self):  
        # Clear all current frames 
        global size,roi1_x, roi1_y, roi1_width, roi1_height,roi2_x, roi2_y, roi2_width, roi2_height
        for widget in self.server_frame.winfo_children():  
            widget.destroy()  
            
        for port, info in self.servers.items():  
            frame = tk.Frame(self.server_frame)  
            frame.pack()  
            

            # Danh sách để lưu thông tin client  
            client_info_list = []  
            for userNamePid in info['clients'].values():  
                if userNamePid is not None:  
                    parts = userNamePid.split('|')  
                    if len(parts) > 1:  # Kiểm tra xem có đủ phần tử không  
                        pid = parts[0]  # pid  
                        username = parts[1]  # username
                        size = f'{parts[2]}x{parts[3]}'
                        if size == '640x480':
                            roi1_x, roi1_y, roi1_width, roi1_height = 35, 10, 30, 100   
                            roi2_x, roi2_y, roi2_width, roi2_height = 100, 10, 30, 100 
                        client_info_list.append(f"{username} (PID: {pid})")  
                        
                        # Thêm thông tin client vào danh sách
            client_info = ', '.join([str(username) for username in info['clients'].values() if username])  
            server_label = tk.Label(frame, text=f"Server on port {port} - Clients: {client_info or 'No clients'}")  
            server_label.pack(side=tk.LEFT)  
            
            for user in info['clients'].values():  
                
                if user is not None:
                        btn = tk.Button(frame, text=f"GetWindow", command=lambda u=user: self.show_window(u))
                        btn.pack(side=tk.LEFT) 
            
            destroy_button = tk.Button(frame, text="Destroy", command=lambda p=port: self.destroy_server(p))  
            destroy_button.pack(side=tk.LEFT)  
    
            

    def destroy_server(self, port):  
        if port in self.servers:  
            server_info = self.servers.pop(port)  
            server_info['server'].shutdown()
            
            # Close associated OpenCV windows
            for client_id, user_info in server_info['clients'].items():
                if user_info:
                    parts = user_info.split('|')
                    if len(parts) > 1:
                        if user_info in self.open_windows:
                            cv2.destroyWindow(user_info)
                            del self.open_windows[user_info]
                                  
            self.update_server_list()     

    def show_window(self, user):
        parts = user.split('|')
        pid = int(parts[0])
        if pid is not None:
            window = self.get_window_by_pid(pid)
            if window is None:  
                print(f"No window found for PID: {pid}. Make sure the process is running.")  
                return
            self.open_windows[user] = True
            thread = threading.Thread(target=self.capture_window, args=(window, user)) 
            thread.daemon = True
            thread.start()

    
    def get_window_by_pid(self, pid):  
        try:  
            process = psutil.Process(pid)  
        except psutil.NoSuchProcess:  
            print(f"No process with PID {pid}")  
            return None  
    
        windows = gw.getAllWindows()  
        
        for window in windows:  
            hwnd = window._hWnd  
            _, window_pid = win32process.GetWindowThreadProcessId(hwnd)  
            
            if window_pid == pid:  
                return window  
    
        print(f"No window found for PID {pid}")  
        return None  

    def send_data_to_client(self, user, elapsed_time):
        # Lấy các phần dựa trên tên cửa sổ  
        for port, info in self.servers.items():  
            for client_id, user_info in info['clients'].items():
                if user == user_info:
                    server = info['server']  
                    message = json.dumps({'username': user, 'elapsed_time': elapsed_time})  
                    server.send_message_to_all(message)  
                    return  # Không cần tiếp tục lặp qua các client khác sau khi đã gửi  
    
    
    def capture_window(self, window, user): 
        global size
        username = user.split('|')[1]
        with mss.mss() as sct:  
            startRun = False  
            elapsed_time = 0  
            start_time = None 
            
            while True:   
                try: 
                    if size != '640x480':
                        monitor = { 
                            "top": window.top + 70,  
                            "left": window.left + 270,  
                            "width": 150,  
                            "height": 100  
                        }  
                    else:
                        monitor = { 
                            "top": window.top + 70,  
                            "left": window.left + 130,  
                            "width": 150,  
                            "height": 100  
                        }  

                    screenshot = sct.grab(monitor)  
                    frame = np.array(screenshot)  
                    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) 
                    
                    # Cắt vùng cần kiểm tra  
                    roi1 = frame[roi1_y:roi1_y + roi1_height, roi1_x:roi1_x + roi1_width]  
                    roi2 = frame[roi2_y:roi2_y + roi2_height, roi2_x:roi2_x + roi2_width]  
            
                    hsv_frame1 = cv2.cvtColor(roi1, cv2.COLOR_BGR2HSV)  
                    mask_light_green1 = cv2.inRange(hsv_frame1, lower_light_green, upper_light_green)   
            
                    hsv_frame2 = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)  
                    mask_light_green2 = cv2.inRange(hsv_frame2, lower_light_green, upper_light_green)   
            
                    contours_light1, _ = cv2.findContours(mask_light_green1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  
                    contours_light2, _ = cv2.findContours(mask_light_green2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  
            
                    current_start_run = False  
                    for contour in contours_light1:  
                        if cv2.contourArea(contour) < 500:  
                            continue  
                        current_start_run = True
                        x, y, w, h = cv2.boundingRect(contour)  
                        cv2.rectangle(roi1, (x, y), (x + w, y + h), (0, 0, 255), 2)  
            
                    current_end_run = False  
                    for contour in contours_light2:  
                        if cv2.contourArea(contour) < 500:  
                            continue  
                        current_end_run = True  
                        x, y, w, h = cv2.boundingRect(contour)  
                        cv2.rectangle(roi2, (x, y), (x + w, y + h), (0, 0, 255), 2)  
            
                    if current_start_run and not startRun:  
                        startRun = True  
                        start_time = time.time()  
                
                    if current_end_run and startRun:  
                        elapsed_time = time.time() - start_time  
                        print(f"{username}: {elapsed_time:.2f} giây")  
                        self.send_data_to_client(user, elapsed_time)
                        startRun = False  
                
                    frame[roi1_y:roi1_y + roi1_height, roi1_x:roi1_x + roi1_width] = roi1  
                    frame[roi2_y:roi2_y + roi2_height, roi2_x:roi2_x + roi2_width] = roi2  
    
                    if user in self.open_windows:
                        cv2.imshow(user, frame)
                    else:
                        break
        
                except Exception as e:  
                    print(f"Lỗi khi chụp ảnh màn hình cho {user}: {e}")  
                    break  
                    
                if cv2.waitKey(1) & 0xFF == ord('q'):   
                    break     
            cv2.destroyAllWindows(user)

    
    def on_closing(self):  
        # Shutdown all servers  
        for server_info in self.servers.values():  
            server_info['server'].shutdown()  
        self.root.destroy()  

if __name__ == "__main__":  
    root = tk.Tk()  
    manager = ServerManager(root)  

    root.mainloop()